# Imports

In [ ]:
import urllib.request
import os
import zipfile
import requests
from bs4 import BeautifulSoup as bs

# Download Data

In [ ]:
URL = "https://covid.saude.gov.br/"
r = requests.get(URL)

In [ ]:
soup = bs(r.text)

In [ ]:
from Naked.toolshed.shell import execute_js, muterun_js

response = muterun_js('https://js-cdn.dynatrace.com/jstag/17052aca8bb/ruxitagent_A2SVfqru_10187200323152418.js')
if response.exitcode == 0:
    print(response.stdout)
else:
    sys.stderr.write(response.stderr)

In [ ]:
https://js-cdn.dynatrace.com/jstag/17052aca8bb/ruxitagent_A2SVfqru_10187200323152418.js

In [ ]:
soup.find_all(attrs={"download": "HIST_PAINEL_COVIDBR_16set2021.zip"})

In [ ]:
# Download zip file
urllib.request.urlretrieve("https://mobileapps.saude.gov.br/esus-vepi/files/unAFkcaNDeXajurGB7LChj8SgQYS2ptm/3d01bb3db1c397423374f53800757630_HIST_PAINEL_COVIDBR_16set2021.zip", '../data/raw/covid_saude/covid_saude.zip')

In [ ]:
# Delete old .csv files
dir_name = '../data/raw/covid_saude'

for item in os.listdir(dir_name): # loop through items in dir
    if item.endswith('.csv'): # check for ".csv" extension
        file_name = os.path.abspath(item) # get full path of files
        file_name = file_name.replace('notebooks', 'data\\raw\\covid_saude')
        os.remove(file_name) # delete zipped file

In [ ]:
# Unzip files and delete zipped file
for item in os.listdir(dir_name): # loop through items in dir
    if item.endswith('.zip'): # check for ".zip" extension
        file_name = os.path.abspath(item) # get full path of files
        file_name = file_name.replace('notebooks', 'data\\raw\\covid_saude')
        zip_ref = zipfile.ZipFile(file_name) # create zipfile object
        zip_ref.extractall(dir_name) # extract file to dir
        zip_ref.close() # close file
        os.remove(file_name) # delete zipped file

# Selenium

In [ ]:
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
import glob,os

In [ ]:
download_path = '../data/raw/covid_saude'
output_path = '../data/raw/covid_saude'

for file in glob.glob(download_path+"*.csv"):
    os.remove(file)

profile = webdriver.FirefoxProfile()
profile.set_preference("browser.download.folderList", 2)
profile.set_preference("browser.download.manager.showWhenStarting",False)
profile.set_preference("browser.download.dir", download_path)
profile.set_preference("browser.helperApps.neverAsk.saveToDisk","text/csv,application/csv,text/plan,text/comma-separated-values")

options = Options()
options.headless = True

In [ ]:
driver=webdriver.Firefox(firefox_profile=profile,options=options)
driver.get("https://covid.saude.gov.br/")

btn = driver.find_element_by_class_name("btn-outline")
btn.click()

driver.close()

# Data Processing

In [ ]:
import pandas, glob

csv_name = glob.glob(download_path+"*.csv")[0]

pd_df = pd.read_csv(csv_name, sep=";")
pd_df = pd_df.rename(columns = lambda x: "data" if "data" in x.lower() else x)
pd_df['data'] = pd_df['data'].astype('datetime64[ns]')
pd_df.to_parquet(output_path+"covid_saude.parquet",header=['regiao','estado','data','casosNovos','casosAcumulados','obitosNovos','obitosAcumulados'], index=False, date_format='%d/%m/%Y')